In [1]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from preprocessing import *
# from grouping_features import grouping 
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:

for i in range(0, len(data)):
    data.loc[i, 'NeighPrice'] = round(data.groupby(['Neighborhood']).median()[['SalePrice']].SalePrice[[data.loc[i, 'Neighborhood']]][0], -4)
    
for i in range(0, len(test)):
    test.loc[i, 'NeighPrice'] = data[data['Neighborhood'] == test.loc[i, 'Neighborhood']]['NeighPrice'].values[0]
    
train=data.sample(frac=0.8,random_state=20)
val=data.drop(train.index)


In [3]:
train = pre_process(train, val, test, True)
val = pre_process(val, None, None, False)
test= pre_process(test, None, None, False)
train, val, test= group(train, val, test)


In [4]:
def rmse(y_pred, y):
    return np.sqrt(np.mean(np.square(y_pred-y)))
def rmsle(y_pred, y):
    return np.sqrt(np.mean(np.square(np.log(y_pred+1) - np.log(y+1))))


In [5]:
train = pd.get_dummies(train)
val = pd.get_dummies(val)
test=pd.get_dummies(test)

In [6]:
Y_train = train['SalePrice'].values
Y_val = val['SalePrice'].values


In [7]:
train.drop(list(set(train) - set(val)), axis=1, inplace = True)
train.drop(list(set(train) - set(test)), axis=1, inplace = True)


In [ ]:
set(val) - set(train)

In [17]:
set(test) - set(train)

{'Condition1_RRNe',
 'ExterCond_Po',
 'Foundation_Stone',
 'Heating_Grav',
 'Heating_Wall',
 'MiscFeature_Gar2',
 'MiscFeature_Othr',
 'RoofStyle_Mansard'}

In [18]:
test.drop(list(set(test) - set(train)), axis = 1, inplace = True)

***Model creation***

In [15]:
train.drop(['Id'], axis = 1, inplace = True)
val.drop(['Id'], axis = 1, inplace = True)
test.drop(['Id'], axis = 1, inplace = True)


val.drop(['SalePrice'], axis = 1, inplace = True)

In [20]:
## assigning a the closest bin to each sale price
rounds = np.histogram(Y_train, bins = 50)[1]
m = np.subtract.outer(Y_train, rounds)
n = np.argmin(abs(m), axis=1)


In [14]:
len(train.columns)

126

In [11]:
train.GarageQual.fillna(0, inplace = True)
val.GarageQual.fillna(0, inplace = True)
test.GarageQual.fillna(0, inplace = True)

***Decision tree***

In [83]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
scx = StandardScaler()
X_train = scx.fit_transform(train)
X_val = scx.transform(val)
X_test = scx.transform(test)

# scy = StandardScaler()
# y_train = scy.fit_transform(Y_train)

# Fitting Decision Tree Classification to the Training set
from sklearn.tree import DecisionTreeRegressor
for i in range(9,12):
    for j in range(9,12):
        regressor = DecisionTreeRegressor(criterion = 'mse', random_state = 0, min_samples_leaf = j, max_leaf_nodes = 50, max_depth = i)
        regressor.fit(X_train, np.log(Y_train))
        Y_train_pred = regressor.predict(X_train)
        Y_val_pred = regressor.predict(X_val)
        print(i, j, rmse(Y_train_pred, np.log(Y_train)), rmse(Y_val_pred, np.log(Y_val)))

9 9 0.134723769959 0.184191383703
9 10 0.135853820381 0.18201997073
9 11 0.137724136127 0.190760854113
10 9 0.134723769959 0.184191383703
10 10 0.135853820381 0.18201997073
10 11 0.137724136127 0.190760854113
11 9 0.134723769959 0.184191383703
11 10 0.135853820381 0.18201997073
11 11 0.137724136127 0.190760854113


***Random forest regression ***

In [92]:
Y_train

array([127500, 324000,  82500, ..., 259000, 144000, 104000])

In [87]:
target = np.vstack((Y_train, Y_val))
observations = np.vstack((X_train, X_val))

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:

from sklearn.ensemble import RandomForestRegressor
for i in range(9,12):
    for j in range(9,12):
        regressor = RandomForestRegressor(criterion = 'mse', random_state = 0, min_samples_leaf = j, max_leaf_nodes = 50, max_depth = i)
        regressor.fit(X_train, np.log(target))
        Y_train_pred = regressor.predict(X_train)
        Y_val_pred = regressor.predict(X_val)
        print(i, j, rmse(Y_train_pred, np.log(target)), rmse(Y_val_pred, np.log(Y_val)))

In [80]:
regressor.tree_

AttributeError: 'sklearn.tree._tree.Tree' object has no attribute 'leaves'

***Predictions***

In [62]:
Y_train_pred = regressor.predict(X_train)

In [63]:
Y_val_pred = regressor.predict(X_val)

Y_test_pred = regressor.predict(X_test)

In [64]:
rmsle(scy.inverse_transform(Y_train_pred), Y_train)

0.21049181005397297

In [65]:
rmsle(scy.inverse_transform(Y_val_pred), Y_val)

0.21980994132684994

***Plots***

In [ ]:
colx = 'Neighborhood'
# print(train.groupby(colx).count()[['Id']]/1460)
fig, ax = plt.subplots(figsize=(15,6))
sns.boxplot(x=colx, y="SalePrice", ax=ax, data=train);

In [ ]:
set(val.select_dtypes(include=['object']).columns) - set(train.select_dtypes(include=['object']).columns )

In [ ]:
train.select_dtypes(exclude=['object']).columns

In [ ]:
plt.scatter(train['MasVnrArea'], train['SalePrice'])

In [ ]:
#missing data
total = train[train.select_dtypes(include=['object']).columns].isnull().sum().sort_values(ascending=False)
percent = (train[train.select_dtypes(include=['object']).columns].isnull().sum()/train[train.select_dtypes(include=['object']).columns].isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Ratio'])
missing_data.head(20)

In [ ]:
#correlation matrix
corrmat = train[train.select_dtypes(exclude=['object']).columns].corr()
f, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(corrmat, square=True, cmap="YlGnBu", linewidths=.5, annot=False);